# Homework 4: Analyzing Rates of Sea Level Rise
In this homework, you'll investigate sea level change using data from the ECCO state estimate. You'll look at spatial differences over the past few decades and then compare how sea level on the California coast compares to the global mean.

Begin by importing the modules you'll need for this notebook:

In [1]:
# import packages here
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc4
import xarray as xr
import os
import glob

## Part 1: Download Sea Surface Height Data
To begin this exercise, download all of the available sea surface height data (1992-2017) from the ECCO Version 4 Release 4 output. There are several different variables for sea surface height pertaining to different components and corrections. In this homework, we'll use the `SSHDYN` variable which is equivalent to what satellites would measure from space.

To obtain the data, use the [Downloading ECCO Data](https://github.com/ProfMikeWood/ocean_modeling_book/blob/main/tools/Downloading%20ECCO%20Data.ipynb) notebook provided with the course notes.

In [2]:
# define the path to the folder where the SSH data has been download on your machine
data_folder = '/Users/benja/SJSU/CS185C/Data'
ssh_folder = os.path.join(data_folder,'ECCO','Version4','Release4', 'interp_monthly','SSHDYN')

## Part 2: Plot Total Sea Level Rise 1992-2017
As a first look into how sea level has changed in the past several decades, we'll look at the differences between two time periods at the beginning and end of the available model output: January 1992 and January 2017. It is important to use the same month because variations in sea level also have a seasonal cycle (as we'll see below). Begin by reading in the data:

In [3]:
# read in the longitude, latitude, and model ssh grid in January 1992 (SSHDYN_1992_01.nc)
ds_jan_1992 = xr.open_dataset(os.path.join(ssh_folder,'SSHDYN_1992_01.nc'))
latitude = ds_jan_1992['latitude'].values
longitude = ds_jan_1992['longitude'].values
ssh_1992 = ds_jan_1992['SSHDYN'].values
ds_jan_1992.close()

# read in the model ssh grid in January 2017 (SSHDYN_2017_01.nc)
ds_jan_2017 = xr.open_dataset(os.path.join(ssh_folder,'SSHDYN_2017_01.nc'))
ssh_2017 = ds_jan_2017['SSHDYN'].values
ds_jan_2017.close()

Now, make a plot of the difference between January 2017 and January 1992. Be sure to add a colorbar with a divergent colormaps and set the color bounds to be equal in the positive and negative directions.

In [4]:
# make a plot of the differences between 2017 and 1992
# Compute sea level difference
SSH_diff = ssh_2017 - ssh_1992

# make a plot of the differences between 2017 and 1992
fig = plt.figure(figsize=(10,6))
C = plt.pcolormesh(longitude,latitude,SSH_diff[0], cmap='seismic', vmin = -.6, vmax= .6)

plt.colorbar(C, orientation='vertical', label='SSHDYN (m)' )
plt.title('SSH Difference\n(1992-2017)')
plt.ylabel('Latitude')
plt.xlabel('Longitude')
plt.tight_layout()
plt.show()

### Questions:
Where are the largest increases in sea level observed between these two time periods? Are there any locations on the globe where sea level has fallen? If so, where?

Answer: *The largest increase in sea level was around 0.5769 m. The sea level fell (blue shades):

West of South America along the equator.
South of Greenland, East of Canada.
West of California and Mexico.
East of Africa.
South of Australia.*

## Part 3: Create a Timeseries of Sea Surface Height in California
Next, we'll make a timeseries of sea surface height at a point off the California coast near San José. A close location in the model is at 37$^{\circ}$N, 122$^{\circ}$W. Begin by finding the closest row and column in the model grid to this longitude and latitude.

In [5]:
# define the latitude and longitude
lon = -122
lat = 37

# find the row and column closest to the location of the point off the CA coast
# note that you have already read in the longitude and latitude if the grid to make the plot above
lat_idx = np.argmin(np.abs(latitude - lat))
lon_idx = np.argmin(np.abs(longitude - lon))

ssh_files = glob.glob(os.path.join(ssh_folder,'SSHDYN_*.nc'))
ssh_files.sort()
print(f'{len(ssh_files)} files')

312 files


Now, armed with your sample location, loop through each sea surface height file to make a timeseries of sea level rise between 1992 and 2017. Every time you access a file, sample the model grid at the row and column you identified above.

In [6]:
# create a timeseries of ssh at the point above by looping through all of the ssh files
# each time through the loop, sample the ssh grid at the row and column identified above
# your timeseries should be an (N,2) array with time in the first column and ssh values in the second
cal_ssh = np.zeros((len(ssh_files),2))

for i, file in enumerate(ssh_files):
    print(f'{i+1} of {len(ssh_files)}', end='\r')
    ds = xr.open_dataset(file)
    cal_ssh[i,0] = ds['time'].values[0]
    cal_ssh[i,1] = ds['SSHDYN'].values[0,lat_idx,lon_idx]
    ds.close()

print(f'\nTimeseries: {cal_ssh.shape}')

312 of 312
Timeseries: (312, 2)


In [7]:
fig = plt.figure(figsize=(10,6))

plt.plot(cal_ssh[:, 0].astype('datetime64[ns]'),cal_ssh[:,1])
plt.title('SSH in California\n(37N, 122W)')
plt.ylabel('SSHDYN (m)')
plt.xlabel('Year')
plt.tight_layout()
plt.show()

## Part 4: Create a Timeseries of Global Mean Sea Surface Height
Next, make a timeseries of sea surface height averaged across the globe. Your loop should look quite similar to that above - you'll just need to make a slight change to average across the whole ocean rather than sampling at a single point.

In [8]:
# create a timeseries of the global mean ssh by looping through all of the ssh files
# each time through the loop, take the mean of the ssh grid at the row and column identified above
# when you take the mean, be sure to exclude 0 values, which pertain to land
# your timeseries should be an (N,2) array with time in the first column and ssh values in the second
global_ssh = np.zeros((len(ssh_files),2))

for i, file in enumerate(ssh_files):
    print(f'{i+1} of {len(ssh_files)}', end='\r')
    ds = xr.open_dataset(file)
    ds = ds.where(ds['SSHDYN'] != 0, drop=True)

    global_ssh[i,0] = ds['time'].values[0]
    global_ssh[i,1] = ds['SSHDYN'].mean(skipna=True).values
    
    ds.close()

print(f'\nTimeseries: {global_ssh.shape}')


312 of 312
Timeseries: (312, 2)


## Part 5: Compute Trends in Sea Surface Height
To compare the two timeseries, compute trends by fitting a line to each timeseries and examining the slope. Print out each trend in mm/yr, rounded to 2 decimal places. Note that the units of sea surface height are in meters.

In [9]:
months = np.arange(len(global_ssh))
dec_yrs = 1992 + months/12

# compute the trends in SLR by fitting a line to the CA and the global mean timeseries
m_cal, b_cal = np.polyfit(dec_yrs, cal_ssh[:,1], 1)
m_glb, b_glb = np.polyfit(dec_yrs, global_ssh[:,1], 1)

print(f'Cal- slope: {m_cal:0.4f}, intercept: {b_cal:0.4f}')
print(f'Gbl- slope: {m_glb:0.4f}, intercept: {b_glb:0.4f}')

# print out the statistics in mm/yr for each timeseries
print(f'{"California:":11} {m_cal*1000:.4f} mm/yr')
print(f'{"Global:":11} {m_glb*1000:.4f} mm/yr')

Cal- slope: 0.0021, intercept: -3.9719
Gbl- slope: 0.0034, intercept: -6.8405
California: 2.0781 mm/yr
Global:     3.3768 mm/yr


### Question:
How does the trend in sea level rise in California compare to the global mean rate?

Answer: *Based on the slope coefficients, the rate of sea level rise in California (2.0781 mm/yr) is approximately 62% of the global mean rate (3.3768 mm/yr). This indicates that while sea levels are rising in California, they are increasing at a slower pace compared to the global average.*

## Part 6: Plot the Timeseries of Sea Surface Height
Finally, plot the two timeseries on the same plot along with their trend lines. Be sure to label your axes and include a legend.

In [12]:
# create your plot here
cal_trend = m_cal * dec_yrs + b_cal
glb_trend = m_glb * dec_yrs + b_glb

plt.figure(figsize=(10, 5))
plt.plot(dec_yrs, cal_ssh[:, 1], label='California SSH', color='blue', alpha=0.5)
plt.plot(dec_yrs, global_ssh[:, 1], label='Global SSH', color='green', alpha=0.5)

plt.plot(dec_yrs, cal_trend, label=f'CA Trend ({m_cal*1000:.2f} mm/yr)', color='blue', linestyle='dashed')
plt.plot(dec_yrs, glb_trend, label=f'Global Trend ({m_glb*1000:.2f} mm/yr)', color='green', linestyle='dashed')

plt.xlabel('Year')
plt.ylabel('Sea Surface Height (m)')
plt.title('Trends in Sea Surface Height (1992-2017)')
plt.legend()
plt.grid(True)
plt.show()

### Questions:

How do the timeseries for Calfiornia and the global mean compare? Comment on their variability. What are the two highest years of sea level in California? What oceanographic phenomenon happened during this time?

Answer: *The California SSH is consistently higher than the global mean SSH, but its rate of increase is slower. Based on the computed trends, California's sea level is rising at approximately 2.08 mm/yr, while the global mean is increasing at about 3.38 mm/yr, meaning California's rate is roughly 62% (or about 2/3) of the global rate. The two highest years of sea level in California occurred around 2015-2016, with another peak in 1998. These peaks coincide with El Niño–Southern Oscillation (ENSO) events, particularly strong El Niño years, which cause temporary but significant rises in sea level due to warmer ocean temperatures, changes in wind patterns, and increased storm activity along the California coast.*